<a href="https://colab.research.google.com/github/boiBASH/Tolaram_Projecct/blob/main/Update_on_Task_for_Tolaram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


# Load data
df = pd.read_csv('/content/drive/MyDrive/Data Analysis - Sample File.csv')

# Clean 'Redistribution Value' and convert to numeric
df['Redistribution Value'] = df['Redistribution Value'].str.replace(',', '', regex=False).astype(float)

# Build user-item matrix
user_item_matrix = df.pivot_table(
    index='Customer_Phone',
    columns='SKU_Code',
    values='Redistribution Value',
    aggfunc='sum'
).fillna(0)

# Encode product features
product_features = df[['SKU_Code', 'Brand']].drop_duplicates().set_index('SKU_Code')
product_features_encoded = pd.get_dummies(product_features, columns=['Brand'])

# Compute similarities
user_sim = cosine_similarity(user_item_matrix)
user_sim_df = pd.DataFrame(user_sim, index=user_item_matrix.index, columns=user_item_matrix.index)

item_sim = cosine_similarity(product_features_encoded)
item_sim_df = pd.DataFrame(item_sim, index=product_features_encoded.index, columns=product_features_encoded.index)

# Hybrid recommendation function
def hybrid_recommend(customer_id, top_n=5):
    sim_users = user_sim_df.loc[customer_id]
    weighted_scores = user_item_matrix.T.dot(sim_users) / sim_users.sum()
    bought = user_item_matrix.loc[customer_id]
    bought_items = bought[bought > 0].index
    content_scores = item_sim_df[bought_items].sum(axis=1)
    final_scores = 0.5 * weighted_scores + 0.5 * content_scores
    final_scores = final_scores.drop(bought_items, errors='ignore')
    rec = final_scores.sort_values(ascending=False).head(top_n).reset_index()
    rec.columns = ['Recommended SKU', 'Score']
    rec['Customer_Phone'] = customer_id
    return rec[['Customer_Phone', 'Recommended SKU', 'Score']]

# Generate recommendations for first 3 customers
sample_customers = user_item_matrix.index[:3]
results = pd.concat([hybrid_recommend(c) for c in sample_customers], ignore_index=True)

results

<ipython-input-2-e34b0ad6faa9>:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Data Analysis - Sample File.csv')


,Customer_Phone,Recommended SKU,Score
0,7010009941,10000539,22160.866123
1,7010009941,10000002,21875.399729
2,7010009941,10002832,18577.795574
3,7010009941,10002861,16795.577580
4,7010009941,10000001,12591.014490
5,7010147753,10000539,29850.173436
6,7010147753,10002678,25267.893307
7,7010147753,10000002,23201.748473
8,7010147753,10002830,18369.500154
9,7010147753,10002857,17409.308841


In [4]:
#clean column
df['Delivered_date'] = pd.to_datetime(df['Delivered_date'], format='%d/%m/%Y %H:%M')

# Compute last purchase date and average inter-purchase interval per customer
grouped = df.sort_values(['Customer_Phone', 'Delivered_date']).groupby('Customer_Phone')
last_dates = grouped['Delivered_date'].last().rename('last_purchase_date')
inter_days = grouped['Delivered_date'].diff().dt.days
avg_days = inter_days.groupby(df['Customer_Phone']).mean().rename('avg_inter_purchase_days')

# Fill single-purchase customers with overall median interval
median_interval = avg_days.median()
avg_days = avg_days.fillna(median_interval)

# Predict next purchase date
summary = pd.concat([last_dates, avg_days], axis=1)
summary['predicted_next_purchase'] = summary['last_purchase_date'] + pd.to_timedelta(summary['avg_inter_purchase_days'], unit='D')

# Determine most frequently purchased SKU per customer as likely next SKU
freq = df.groupby(['Customer_Phone', 'SKU_Code']).size().rename('count').reset_index()
idx = freq.groupby('Customer_Phone')['count'].idxmax()
likely = freq.loc[idx, ['Customer_Phone', 'SKU_Code']].rename(columns={'SKU_Code': 'likely_next_SKU'})

# Merge and reset index
summary = summary.reset_index().merge(likely, on='Customer_Phone')

summary

,Customer_Phone,last_purchase_date,avg_inter_purchase_days,predicted_next_purchase,likely_next_SKU
0,7010009941,2025-01-02,15.555556,2025-01-17 13:20:00.000000003,10002866
1,7010147753,2025-01-29,11.923077,2025-02-09 22:09:13.846153844,10002805
2,7010301833,2025-01-06,5.818182,2025-01-11 19:38:10.909090907,10003376
3,7010572000,2025-01-29,3.469388,2025-02-01 11:15:55.102040812,10002861
4,7010755909,2025-01-31,5.375000,2025-02-05 09:00:00.000000000,10000001
...,...,...,...,...,...
4006,9166785982,2025-01-15,14.727273,2025-01-29 17:27:16.363636361,10002849
4007,9166958284,2025-01-29,17.500000,2025-02-15 12:00:00.000000000,10002976
4008,9166975415,2025-01-23,2.692308,2025-01-25 16:36:55.384615385,10002866
4009,9167401267,2025-01-14,9.529412,2025-01-23 12:42:21.176470589,10002832


In [10]:
# Define max date for recency calculation
max_date = df['Delivered_date'].max()

# Calculate RFM metrics
# Recency: days since last purchase
last_purchase = df.groupby('Customer_Phone')['Delivered_date'].max()
recency = (max_date - last_purchase).dt.days

# Frequency: number of unique orders
frequency = df.groupby('Customer_Phone')['Order_Id'].nunique()

# Monetary: average order value per customer
order_values = df.groupby(['Customer_Phone', 'Order_Id'])['Redistribution Value'].sum().reset_index()
monetary = order_values.groupby('Customer_Phone')['Redistribution Value'].mean()

# Assemble RFM DataFrame
rfm = pd.DataFrame({
    'Recency': recency,
    'Frequency': frequency,
    'Monetary': monetary
})

# Calculate quantile thresholds
recency_q = rfm['Recency'].quantile([0.25, 0.5, 0.75])
frequency_q = rfm['Frequency'].quantile([0.25, 0.5, 0.75])
monetary_q = rfm['Monetary'].quantile([0.25, 0.5, 0.75])

# Define segmentation logic
def rfm_segment(row):
    if row['Recency'] <= recency_q.loc[0.25] and row['Frequency'] >= frequency_q.loc[0.75] and row['Monetary'] >= monetary_q.loc[0.75]:
        return 'Best Customers'
    elif row['Recency'] >= recency_q.loc[0.75] and row['Frequency'] <= frequency_q.loc[0.25]:
        return 'At-Risk Customers'
    elif row['Recency'] >= recency_q.loc[0.75] and row['Monetary'] >= monetary_q.loc[0.75]:
        return 'Big Spenders Dropping Off'
    elif recency_q.loc[0.25] < row['Recency'] <= recency_q.loc[0.75] and row['Frequency'] >= frequency_q.loc[0.5]:
        return 'Potential Loyalists'
    else:
        return 'Others'

# Apply segmentation
rfm['Segment'] = rfm.apply(rfm_segment, axis=1)

# Count customers in each segment
segment_counts = rfm['Segment'].value_counts().reset_index()
segment_counts.columns = ['Segment', 'Number of Customers']

segment_counts

,Segment,Number of Customers
0,Others,2083
1,Potential Loyalists,1071
2,At-Risk Customers,608
3,Best Customers,138
4,Big Spenders Dropping Off,111


In [12]:
# Calculate RFM metrics
max_date = df['Delivered_date'].max()
last_purchase = df.groupby('Customer_Phone')['Delivered_date'].max()
recency = (max_date - last_purchase).dt.days
frequency = df.groupby('Customer_Phone')['Order_Id'].nunique()
order_values = df.groupby(['Customer_Phone', 'Order_Id'])['Redistribution Value'].sum().reset_index()
monetary = order_values.groupby('Customer_Phone')['Redistribution Value'].mean()

rfm = pd.DataFrame({
    'Recency': recency,
    'Frequency': frequency,
    'Monetary': monetary
})

# Quantile thresholds
recency_q = rfm['Recency'].quantile([0.25, 0.5, 0.75])
frequency_q = rfm['Frequency'].quantile([0.25, 0.5, 0.75])
monetary_q = rfm['Monetary'].quantile([0.25, 0.5, 0.75])

# Segmentation logic
def rfm_segment(row):
    if row['Recency'] <= recency_q.loc[0.25] and row['Frequency'] >= frequency_q.loc[0.75] and row['Monetary'] >= monetary_q.loc[0.75]:
        return 'Best Customers'
    elif row['Recency'] >= recency_q.loc[0.75] and row['Frequency'] <= frequency_q.loc[0.25]:
        return 'At-Risk Customers'
    elif row['Recency'] >= recency_q.loc[0.75] and row['Monetary'] >= monetary_q.loc[0.75]:
        return 'Big Spenders Dropping Off'
    elif recency_q.loc[0.25] < row['Recency'] <= recency_q.loc[0.75] and row['Frequency'] >= frequency_q.loc[0.5]:
        return 'Potential Loyalists'
    else:
        return 'Others'

rfm['Segment'] = rfm.apply(rfm_segment, axis=1)

# Discount recommendations for Best and Loyal segments
median_best = rfm.loc[rfm['Segment'] == 'Best Customers', 'Monetary'].median()
median_loyal = rfm.loc[rfm['Segment'] == 'Potential Loyalists', 'Monetary'].median()

rfm['Recommended_Discount'] = 0
rfm.loc[(rfm['Segment'] == 'Best Customers') & (rfm['Monetary'] < median_best), 'Recommended_Discount'] = 10
rfm.loc[(rfm['Segment'] == 'Potential Loyalists') & (rfm['Monetary'] < median_loyal), 'Recommended_Discount'] = 5

# Prepare output
discounts = (
    rfm[rfm['Recommended_Discount'] > 0]
    .reset_index()
    .rename(columns={'index': 'Customer_Phone', 'Monetary': 'Avg_Spend'})
    [['Customer_Phone', 'Segment', 'Avg_Spend', 'Recommended_Discount']]
)

discounts

,Customer_Phone,Segment,Avg_Spend,Recommended_Discount
0,7010919799,Potential Loyalists,33757.500000,5
1,7011543993,Potential Loyalists,25244.444444,5
2,7012439844,Potential Loyalists,39499.193548,5
3,7014463034,Potential Loyalists,35083.333333,5
4,7015755589,Potential Loyalists,34364.705882,5
...,...,...,...,...
599,9130039218,Potential Loyalists,41351.923077,5
600,9131118038,Best Customers,84306.060606,10
601,9134997275,Potential Loyalists,43585.000000,5
602,9137148847,Potential Loyalists,46372.058824,5
